In [1]:
import numpy as np
from astropy.io import fits
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from skimage import exposure, filters
from astropy.convolution import convolve, Gaussian2DKernel
from tensorflow.keras.applications.vgg16 import preprocess_input


2024-09-06 22:26:14.891108: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-06 22:26:14.987134: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-06 22:26:15.015725: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-06 22:26:15.218609: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-06 22:26:16.301896: W tensorflow/compiler/tf2

In [4]:


# Path to the FITS file
fits_file_path = 'space images/output_images_with_labels.fits'

# Lists to hold image data and labels
image_data_list = []
label_list = []

# Desired image size
# target_size = (256, 256)

# Open the FITS file
with fits.open(fits_file_path) as hdulist:
    # Iterate over each ImageHDU (starting from index 1)
    for hdu in hdulist[1:-1]:  # Skip the primary HDU (index 0) and the last HDU (label table)
        if isinstance(hdu, fits.ImageHDU):
            # Extract image data
            image_data = hdu.data
            
            # Convert the image data to PIL Image for resizing
            image = Image.fromarray(image_data)

            # Resize the image to 256x256
            # resized_image = image.resize(target_size)

            # Convert the resized image back to NumPy array
            resized_image_data = np.array(image)

            # Append the resized image data and corresponding label to the lists
            image_data_list.append(resized_image_data)
            label_list.append(hdu.header.get('OBJECT'))

# Convert the lists to NumPy arrays
image_data_array = np.array(image_data_list)
label_array = np.array(label_list)

# Encode labels as integers
# label_encoder = LabelEncoder()
# encoded_labels = label_encoder.fit_transform(label_array)

# Verify the shapes of the arrays
print(f"Resized image data array shape: {image_data_array.shape}")
# print(f"Encoded labels: {encoded_labels}")
print(f"Encoded labels: {label_array}")
set(label_array)

Resized image data array shape: (1683, 128, 128, 3)
Encoded labels: ['cosmos space' 'cosmos space' 'cosmos space' ... 'stars' 'stars' 'stars']


{'Black Holes',
 'constellation',
 'cosmos space',
 'galaxies',
 'nebula',
 'planets',
 'stars'}

In [5]:
import numpy as np
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from sklearn.model_selection import train_test_split


In [6]:

# Assuming `image_data_array` and `label_array` are already defined
# Encode labels as integers
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(label_array)

# One-hot encode the labels for categorical cross-entropy loss
one_hot_labels = to_categorical(encoded_labels)

# image_data_array = np.expand_dims(image_data_array, axis=-1)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(image_data_array, one_hot_labels, test_size=0.2, random_state=42)


In [7]:

# Load VGG16 model with pre-trained weights and exclude the top fully connected layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Add custom top layers
x = base_model.output
x = Flatten()(x)
# x = Dropout(0.2)(x)
x = BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
# x = Dense(36, activation='relu')(x)
predictions = Dense(len(np.unique(encoded_labels)), activation='softmax')(x)

# Create model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


2024-09-06 22:26:54.012510: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: UNKNOWN ERROR (100)


In [8]:

# Prepare data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Train the model
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# # Define the callbacks
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

# Define batch size
batch_size = 4

# Train the model with data augmentation and callbacks
model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
          steps_per_epoch=len(X_train) // batch_size,
          epochs=25,
          validation_data=(X_test, y_test))
        #   callbacks=[early_stopping, model_checkpoint])


Epoch 1/25


/home/hardik/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


336/336 ━━━━━━━━━━━━━━━━━━━━ 40s 116ms/step - accuracy: 0.4124 - loss: 2.8088 - val_accuracy: 0.6142 - val_loss: 2.5182
Epoch 2/25
  1/336 ━━━━━━━━━━━━━━━━━━━━ 43s 131ms/step - accuracy: 0.2500 - loss: 2.3413

2024-09-06 22:27:42.565236: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


336/336 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.2500 - loss: 2.3413 - val_accuracy: 0.6172 - val_loss: 2.5073
Epoch 3/25
336/336 ━━━━━━━━━━━━━━━━━━━━ 39s 116ms/step - accuracy: 0.5187 - loss: 1.6846 - val_accuracy: 0.6677 - val_loss: 2.2332
Epoch 4/25
  1/336 ━━━━━━━━━━━━━━━━━━━━ 33s 99ms/step - accuracy: 0.2500 - loss: 2.3017

...

336/336 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.2500 - loss: 2.3017 - val_accuracy: 0.6647 - val_loss: 2.2317
Epoch 5/25
336/336 ━━━━━━━━━━━━━━━━━━━━ 74s 112ms/step - accuracy: 0.5761 - loss: 1.2601 - val_accuracy: 0.6736 - val_loss: 2.1044
Epoch 6/25
336/336 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - accuracy: 0.2500 - loss: 1.2307 - val_accuracy: 0.6736 - val_loss: 2.1139
Epoch 7/25
336/336 ━━━━━━━━━━━━━━━━━━━━ 38s 113ms/step - accuracy: 0.5888 - loss: 1.2152 - val_accuracy: 0.6795 - val_loss: 2.0935
Epoch 8/25
  1/336 ━━━━━━━━━━━━━━━━━━━━ 32s 96ms/step - accuracy: 0.7500 - loss: 1.0134

2024-09-06 22:30:33.532698: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


336/336 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.7500 - loss: 1.0134 - val_accuracy: 0.6766 - val_loss: 2.1108
Epoch 9/25
336/336 ━━━━━━━━━━━━━━━━━━━━ 42s 125ms/step - accuracy: 0.5711 - loss: 1.1734 - val_accuracy: 0.6944 - val_loss: 2.1964
Epoch 10/25
336/336 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 0.7500 - loss: 0.6346 - val_accuracy: 0.6973 - val_loss: 2.1955
Epoch 11/25
336/336 ━━━━━━━━━━━━━━━━━━━━ 39s 117ms/step - accuracy: 0.5985 - loss: 1.1156 - val_accuracy: 0.7181 - val_loss: 1.8820
Epoch 12/25
336/336 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.2500 - loss: 1.8566 - val_accuracy: 0.7122 - val_loss: 1.8787
Epoch 13/25
336/336 ━━━━━━━━━━━━━━━━━━━━ 39s 116ms/step - accuracy: 0.6264 - loss: 1.0992 - val_accuracy: 0.6884 - val_loss: 2.1413
Epoch 14/25
336/336 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.7500 - loss: 3.7667 - val_accuracy: 0.6973 - val_loss: 2.1358
Epoch 15/25
336/336 ━━━━━━━━━━━━━━━━━━━━ 36s 106ms/step - accuracy: 0.6076 - loss: 1.0877 - val_a

2024-09-06 22:33:37.072173: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


336/336 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.7500 - loss: 0.8423 - val_accuracy: 0.6825 - val_loss: 2.2199
Epoch 17/25
336/336 ━━━━━━━━━━━━━━━━━━━━ 37s 110ms/step - accuracy: 0.5441 - loss: 1.2278 - val_accuracy: 0.6736 - val_loss: 2.3350
Epoch 18/25
336/336 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.0000e+00 - loss: 2.2624 - val_accuracy: 0.6706 - val_loss: 2.3455
Epoch 19/25
336/336 ━━━━━━━━━━━━━━━━━━━━ 38s 114ms/step - accuracy: 0.6095 - loss: 1.0948 - val_accuracy: 0.6706 - val_loss: 1.9441
Epoch 20/25
336/336 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - accuracy: 0.5000 - loss: 2.2371 - val_accuracy: 0.6706 - val_loss: 1.9350
Epoch 21/25
336/336 ━━━━━━━━━━━━━━━━━━━━ 38s 114ms/step - accuracy: 0.6247 - loss: 1.0408 - val_accuracy: 0.6736 - val_loss: 2.3247
Epoch 22/25
336/336 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.7500 - loss: 0.4773 - val_accuracy: 0.6736 - val_loss: 2.3257
Epoch 23/25
336/336 ━━━━━━━━━━━━━━━━━━━━ 38s 114ms/step - accuracy: 0.6519 - loss: 1.0163 - 

In [9]:
model.save('Classifier.h5')


In [10]:

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")


11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 550ms/step - accuracy: 0.6778 - loss: 2.1629
Test accuracy: 0.6884


In [12]:
def predict_object_type(img_path, model, label_encoder):
    # Load and preprocess the image using PIL
    img = Image.open(img_path)
    img = img.resize((128, 128))  # Resize to match the model input shape
    img_array = np.array(img)  # Convert PIL image to numpy array
    
    # Ensure the image has 3 channels (RGB), convert if needed
    if img_array.shape[-1] != 3:
        img_array = np.stack([img_array] * 3, axis=-1)
    
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = preprocess_input(img_array)  # Preprocess the image as required by VGG16

    # Predict the object type
    predictions = model.predict(img_array)
    predicted_class_idx = np.argmax(predictions, axis=1)

    # Convert the predicted index to the original label
    predicted_label = label_encoder.inverse_transform(predicted_class_idx)
    
    print(f"Predicted object: {predicted_label[0]}")
    return predicted_label[0]


In [19]:
model.load('Classifier.h5')
# x=predict_object_type(r'space images/galaxies/12.jpg',model,label_encoder)
# print(x)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Predicted object: stars
stars


In [11]:


def enhance_color_bands(fits_files, object_type):
    """
    Enhance and combine color bands into a single RGB image.

    :param fits_files: List of paths to FITS files corresponding to different bands.
    :param object_type: Type of astronomical object (e.g., 'black_hole', 'galaxy', 'nebula', 'star').
    :return: RGB image array.
    """
    bands = [fits.getdata(f) for f in fits_files]
    if object_type == 'black_hole':
        return enhance_black_hole_color(bands)
    elif object_type == 'constellation':
        return enhance_constellation_color(bands)
    elif object_type == 'galaxy':
        return enhance_galaxy_color(bands)
    elif object_type == 'nebula':
        return enhance_nebula_color(bands)
    elif object_type == 'cosmos':
        return enhance_cosmos_color(bands)
    elif object_type == 'planet':
        return enhance_planet_color(bands)
    elif object_type == 'star':
        return enhance_star_color(bands)
    else:
        raise ValueError("Unknown object type")

def preprocess_band(band):
    """
    Preprocess a single band: apply normalization and contrast stretching.

    :param band: 2D array of the FITS band.
    :return: Processed 2D array.
    """
    band = exposure.rescale_intensity(band, in_range='image', out_range=(0, 1))
    band = exposure.equalize_hist(band)
    return band

def enhance_black_hole_color(bands):
    """
    Enhance and combine color bands for a black hole.
    """
    red, green, blue = [preprocess_band(band) for band in bands]

    # Combine bands into an RGB image
    rgb_image = np.dstack([red, green, blue])
    return rgb_image

def enhance_constellation_color(bands):
    """
    Enhance and combine color bands for star constellations.
    """
    red, green, blue = [preprocess_band(band) for band in bands]
    red = filters.unsharp_mask(red, radius=1, amount=2)
    green = filters.unsharp_mask(green, radius=1, amount=2)
    blue = filters.unsharp_mask(blue, radius=1, amount=2)
    rgb_image = np.dstack([red, green, blue])
    return rgb_image

def enhance_galaxy_color(bands):
    """
    Enhance and combine color bands for a galaxy.
    """
    red, green, blue = [preprocess_band(band) for band in bands]
    red = filters.unsharp_mask(red, radius=1, amount=1.5)
    green = filters.unsharp_mask(green, radius=1, amount=1.5)
    blue = filters.unsharp_mask(blue, radius=1, amount=1.5)
    rgb_image = np.dstack([red, green, blue])
    return rgb_image

def enhance_nebula_color(bands):
    """
    Enhance and combine color bands for a nebula.
    """
    red, green, blue = [preprocess_band(band) for band in bands]
    red = filters.unsharp_mask(red, radius=1, amount=2)
    green = filters.unsharp_mask(green, radius=1, amount=2)
    blue = filters.unsharp_mask(blue, radius=1, amount=2)
    rgb_image = np.dstack([red, green, blue])
    return rgb_image

def enhance_cosmos_color(bands):
    """
    Enhance and combine color bands for a wide-field cosmos image.
    """
    red, green, blue = [preprocess_band(band) for band in bands]
    kernel = Gaussian2DKernel(x_stddev=1)
    red_smooth = convolve(red, kernel)
    green_smooth = convolve(green, kernel)
    blue_smooth = convolve(blue, kernel)
    rgb_image = np.dstack([red_smooth, green_smooth, blue_smooth])
    return rgb_image

def enhance_planet_color(bands):
    """
    Enhance and combine color bands for a planet.
    """
    red, green, blue = [preprocess_band(band) for band in bands]
    red = filters.unsharp_mask(red, radius=1, amount=1.5)
    green = filters.unsharp_mask(green, radius=1, amount=1.5)
    blue = filters.unsharp_mask(blue, radius=1, amount=1.5)
    rgb_image = np.dstack([red, green, blue])
    return rgb_image

def enhance_star_color(bands):
    """
    Enhance and combine color bands for star fields.
    """
    red, green, blue = [preprocess_band(band) for band in bands]
    red_filtered = filters.unsharp_mask(red, radius=1, amount=2)
    green_filtered = filters.unsharp_mask(green, radius=1, amount=2)
    blue_filtered = filters.unsharp_mask(blue, radius=1, amount=2)
    rgb_image = np.dstack([red_filtered, green_filtered, blue_filtered])
    return rgb_image

def display_image(image, title="Enhanced Image"):
    """
    Display an RGB image using matplotlib.

    :param image: 3D array of the RGB image.
    :param title: Title of the plot.
    """
    plt.figure(figsize=(10, 10))
    plt.imshow(image, origin='lower')
    plt.title(title)
    plt.axis('off')
    plt.show()


In [12]:

def output_final(fits_file,model,label_encoder):
    """
    Split a single FITS file into three-band FITS files: Red, Green, and Blue.

    :param fits_file: Path to the single-band FITS file.
    :return: List of paths to the new Red, Green, and Blue FITS files.
    """
    # Load the single-band FITS file
    with fits.open(fits_file) as hdul:
        data = hdul[0].data
    
    # Check the data shape and type
    if data.ndim != 2:
        raise ValueError("The FITS file must contain a 2D array.")
    
    # Create three bands (red, green, blue) from the original data
    # Here, we're simply duplicating the original data for all three bands
    # In a real scenario, you might want to use different data or apply transformations
    red_band = data
    green_band = data
    blue_band = data
    
    # Define paths for the new FITS files
    red_path = fits_file.replace('.fits', '_red.fits')
    green_path = fits_file.replace('.fits', '_green.fits')
    blue_path = fits_file.replace('.fits', '_blue.fits')

    # Save the bands as separate FITS files
    fits.writeto(red_path, red_band, overwrite=True)
    fits.writeto(green_path, green_band, overwrite=True)
    fits.writeto(blue_path, blue_band, overwrite=True)

    if data.ndim != 2:
        raise ValueError("The FITS file must contain a 2D array.")
    
    # Normalize data to [0, 1]
    data_min = np.min(data)
    data_max = np.max(data)
    normalized_data = (data - data_min) / (data_max - data_min)
    
    # Convert normalized data to [0, 255] for saving as image
    image_data = (normalized_data * 255).astype(np.uint8)

    object=predict_object_type(image_data,model,label_encoder)
    fits_files=[red_path,green_path,blue_path]
    enhanced_image=enhance_color_bands(fits_files,object)
    display_image(enhanced_image, title=f"Enhanced {object.replace('_', ' ').capitalize()} Image")

    return enhanced_image,object

